Gomoku Info:
    Black starts
    1 is Black
    0 is White

    0 : TurnColor
    1-7 : WhiteHistory
    8-14: BlackHistory

    History is lower index -> newer position

In [1]:
import numpy as np
import torch
import torch.nn as nn
import time
from IPython.display import clear_output

In [4]:
class ResidualLayer(nn.Module):
    def __init__(self, filters, kernal_size=3):
        super().__init__()

        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
        )

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.conv2d_sequential(x)
        x += residual
        x = self.relu(x)

        return x
    
class ConvolutionLayer(nn.Module):
    def __init__(self, infilters, outfilters, kernal_size=3):
        super().__init__()
        
        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(infilters, outfilters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(outfilters),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.conv2d_sequential(x)
        return x
    
class PolicyHead(nn.Module):
    def __init__(self, filters):
        super().__init__()
        self.filters = filters

        self.head = nn.Sequential(
            nn.Conv2d(self.filters, 1, 1),
            nn.Flatten(),
            nn.BatchNorm1d(225),
            nn.ReLU(),
            nn.Linear(225, 225)
        )

    def forward(self, x):
        x = self.head(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, filters, feature_dimensions, residual_layers=5, kernal_size=3):
        super().__init__()

        self.conv_layer = ConvolutionLayer(feature_dimensions, filters, kernal_size=kernal_size)
        self.residual_layers = nn.ModuleList([ResidualLayer(filters, kernal_size=kernal_size) for _ in range(residual_layers)])
        self.policy_head = PolicyHead(filters)

    def forward(self, x):      
        x = self.conv_layer(x)
        for layer in self.residual_layers:
            x = layer(x)
        x = self.policy_head(x)
        
        return x
    
Filters = 128
Layers = 20
HistoryDepth = 7
KernalSize = 5

model = NeuralNetwork(Filters, HistoryDepth * 2 + 1, Layers, kernal_size=KernalSize)
#model = torch.compile(model)

In [4]:
model.load_state_dict(torch.load('../Models/HumanModels/128f20l5kAllBut715e/14.pt'))
model.eval()

OptimizedModule(
  (_orig_mod): NeuralNetwork(
    (conv_layer): ConvolutionLayer(
      (conv2d_sequential): Sequential(
        (0): Conv2d(15, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (residual_layers): ModuleList(
      (0-39): 40 x ResidualLayer(
        (conv2d_sequential): Sequential(
          (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU()
      )
    )
    (policy_head): PolicyHead(
      (head): Sequential(
        (0): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
        (1): 

In [5]:
def makeMove(gamestate, x, y):
    turn = gamestate[0, 0, 0]
    whiteHistory = []
    blackHistory = []

    if HistoryDepth == 1:
        if turn:
            gamestate[2, x, y] = True
        else:
            gamestate[1, x, y] = True
        gamestate[0, :, :] = not turn
        return gamestate

    for i in range(HistoryDepth):
        whiteHistory.append(gamestate[i + 1].copy())
        blackHistory.append(gamestate[i + HistoryDepth + 1].copy())
    currentWhite = whiteHistory[0].copy()
    currentBlack = blackHistory[0].copy()

    if turn:
        currentBlack[x, y] = True
    else:
        currentWhite[x, y] = True
    
    gamestate[0, :, :] = not turn
    gamestate[1] = currentWhite
    gamestate[HistoryDepth + 1] = currentBlack

    for i in range(1, HistoryDepth):
        gamestate[i + 1] = whiteHistory[i - 1]
    for i in range(1, HistoryDepth):
        gamestate[i + HistoryDepth + 1] = blackHistory[i - 1]

    return gamestate

def renderGamestateSlice(gamestate, depth):
    if HistoryDepth == 1:
        blackStones = gamestate[2][:][:]
        whiteStones = gamestate[1][:][:]
    else:
        blackStones = gamestate[HistoryDepth * 2 - depth][:][:]
        whiteStones = gamestate[HistoryDepth - depth][:][:]
    print("     1   2   3   4   5   6   7   8   9   10  11  12  13  14  15")
    print("   --------------------------------------------------------------")
    for y in range(15):
        print(f'{15 - y:2} |', end="")
        for x in range(15):
            if blackStones[x][y] == 0 and whiteStones[x][y] == 0:
                print("   ", end="")
            elif blackStones[x][y] == 1:
                print(" X ", end="")
            elif whiteStones[x][y] == 1:
                print(" O ", end="")
            print("|", end="")
        print("\n   --------------------------------------------------------------")

In [6]:
board = np.zeros((HistoryDepth * 2 + 1, 15, 15), dtype=bool)
board[0, :, :] = True

In [7]:
while (True):
    modelInput = torch.from_numpy(board.astype(np.float32)).unsqueeze(0)
    with torch.no_grad():
        output = model(modelInput)
    output = torch.nn.functional.softmax(output, dim=1)
    output = np.array(output)
    output = output.reshape(15, 15)
    output *= ~board[1]
    output *= ~board[HistoryDepth + 1]
    output = output.flatten()

    index = output.argmax()
    x, y = int(index // 15), int(index % 15)

    board = makeMove(board, x, y)
    clear_output(wait=True)
    renderGamestateSlice(board, 6)
    time.sleep(1)

    x = int(input("X:"))
    y = int(input("Y:"))

    board = makeMove(board, x - 1, 14 - y + 1)
    clear_output(wait=True)
    renderGamestateSlice(board, 6)
    time.sleep(1)

     1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
   --------------------------------------------------------------
15 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
14 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
13 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
12 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
11 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
10 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 9 |   |   |   |   |   |   | X |   |   |   |   |   |   |   |   |
   ----------------

ValueError: invalid literal for int() with base 10: ''